In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
print('Loading init weather data...')
# load and concatenate weather data
weather_dtypes = {
    'site_id': np.uint8,
    'air_temperature': np.float32,
    'cloud_coverage': np.float32,
    'dew_temperature': np.float32,
    'precip_depth_1_hr': np.float32,
    'sea_level_pressure': np.float32,
    'wind_direction': np.float32,
    'wind_speed': np.float32,
}

weather_train = pd.read_csv(
    'weather_train.csv',
    dtype=weather_dtypes,
    parse_dates=['timestamp']
)
weather_test = pd.read_csv(
    'weather_test.csv',
    dtype=weather_dtypes,
    parse_dates=['timestamp']
)

weather = pd.concat(
    [
        weather_train,
        weather_test
    ],
    ignore_index=True
)
# del redundant dfs
del weather_train, weather_test

weather.head()

Loading init weather data...


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.000000,6.0,20.0,NaN,1019.700012,0.0,0.0
1,0,2016-01-01 01:00:00,24.400000,NaN,21.1,-1.0,1020.200012,70.0,1.5
2,0,2016-01-01 02:00:00,22.799999,2.0,21.1,0.0,1020.200012,0.0,0.0
3,0,2016-01-01 03:00:00,21.100000,2.0,20.6,0.0,1020.099976,0.0,0.0
4,0,2016-01-01 04:00:00,20.000000,2.0,20.0,-1.0,1020.000000,250.0,2.6


In [3]:
weather_key = ['site_id', 'timestamp']
temp_skeleton = weather[weather_key + ['air_temperature']]\
.drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()
# check sample
temp_skeleton.head()

,site_id,timestamp,air_temperature
0,0,2016-01-01 00:00:00,25.000000
1,0,2016-01-01 01:00:00,24.400000
2,0,2016-01-01 02:00:00,22.799999
3,0,2016-01-01 03:00:00,21.100000
4,0,2016-01-01 04:00:00,20.000000


In [5]:
# calculate ranks of hourly temperatures within date/site_id chunks
temp_skeleton['temp_rank'] = temp_skeleton.groupby(
    ['site_id', temp_skeleton.timestamp.dt.date],
)['air_temperature'].rank('average')
temp_skeleton

,site_id,timestamp,air_temperature,temp_rank
0,0,2016-01-01 00:00:00,25.000000,16.0
1,0,2016-01-01 01:00:00,24.400000,15.0
2,0,2016-01-01 02:00:00,22.799999,13.0
3,0,2016-01-01 03:00:00,21.100000,9.0
4,0,2016-01-01 04:00:00,20.000000,3.5
5,0,2016-01-01 05:00:00,19.400000,2.0
6,0,2016-01-01 06:00:00,21.100000,9.0
7,0,2016-01-01 07:00:00,21.100000,9.0
8,0,2016-01-01 08:00:00,20.600000,5.5
9,0,2016-01-01 09:00:00,21.100000,9.0


In [13]:
# create 2D dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = temp_skeleton.groupby(
    ['site_id', temp_skeleton.timestamp.dt.hour]
)['temp_rank'].mean().unstack(level=1)
df_2d

timestamp,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
site_id,,,,,,,,,,,,,,,,,,,,,
0,15.123175,13.351277,11.868157,10.541971,9.299270,7.907391,6.897354,5.782847,5.042922,4.430594,...,14.081661,16.429745,18.217609,19.564325,20.555201,21.162409,21.013686,19.938412,18.085310,15.528741
1,8.950691,7.786636,6.781049,5.786175,5.062557,4.993560,5.914443,7.679853,9.504604,12.055249,...,21.207948,21.104533,20.023105,18.373728,16.498612,14.616913,12.806377,11.278855,9.765869,8.133180
2,21.754562,20.260493,18.296804,16.325730,14.544252,12.886405,11.361314,10.010036,8.574361,7.144617,...,2.723084,5.034672,8.709398,11.976277,14.525091,16.870894,18.868613,20.568493,21.755474,22.093978
3,16.497715,14.732176,13.162409,11.897717,10.696347,9.474406,8.282192,7.164384,6.184307,5.220347,...,11.579836,13.933850,15.972172,17.749088,19.243613,20.200274,20.352511,19.704566,17.960310,15.894521
4,20.952425,19.079836,16.622719,14.306569,12.331661,10.831661,9.597628,8.402372,7.549360,6.820255,...,4.696624,6.607664,9.773266,13.172445,15.905566,18.035584,19.823449,21.186588,21.803832,21.501369
5,9.062095,8.491220,7.997692,7.461716,6.926199,6.691705,7.049815,8.276959,9.910978,12.172509,...,20.143514,19.839649,18.599537,17.022706,15.252089,13.423006,11.604898,10.344588,9.551105,8.967281
6,16.386093,14.572536,13.068431,11.704296,10.390768,9.168037,7.946527,6.844607,5.966667,5.209324,...,11.722831,14.281279,16.205936,17.940693,19.468037,20.508219,20.876832,20.161792,18.289954,15.754338
7,16.195570,14.469885,12.982364,11.843602,10.484630,9.453200,8.366509,7.306775,6.379278,5.549575,...,11.145793,13.195714,15.047115,16.580676,18.130332,19.243528,19.773206,19.385052,17.969466,16.013889
8,15.123175,13.351277,11.868157,10.541971,9.299270,7.907391,6.897354,5.782847,5.042922,4.430594,...,14.081661,16.429745,18.217609,19.564325,20.555201,21.162409,21.013686,19.938412,18.085310,15.528741


In [14]:
# align scale, so each value within row is in [0,1] range
df_2d = df_2d / df_2d.max(axis=1).values.reshape((-1,1))  
df_2d

timestamp,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
site_id,,,,,,,,,,,,,,,,,,,,,
0,0.714624,0.630896,0.560813,0.498146,0.439424,0.373653,0.325925,0.273260,0.238296,0.209361,...,0.665409,0.776365,0.860848,0.924485,0.971307,1.000000,0.992972,0.942162,0.854596,0.733789
1,0.422044,0.367156,0.319741,0.272830,0.238710,0.235457,0.278879,0.362121,0.448162,0.568431,...,1.000000,0.995124,0.944132,0.866360,0.777945,0.689219,0.603848,0.531822,0.460482,0.383497
2,0.984638,0.917014,0.828135,0.738922,0.658290,0.583254,0.514227,0.453066,0.388086,0.323374,...,0.123250,0.227875,0.394198,0.542061,0.657423,0.763597,0.854016,0.930955,0.984679,1.000000
3,0.810598,0.723850,0.646722,0.584582,0.525554,0.465515,0.406937,0.352015,0.303860,0.256496,...,0.568963,0.684626,0.784776,0.872083,0.945515,0.992520,1.000000,0.968164,0.882462,0.780961
4,0.960951,0.875068,0.762376,0.656149,0.565573,0.496778,0.440181,0.385362,0.346240,0.312801,...,0.215404,0.303051,0.448236,0.604134,0.729485,0.827175,0.909173,0.971691,1.000000,0.986128
5,0.449877,0.421536,0.397036,0.370428,0.343843,0.332201,0.349979,0.410899,0.492018,0.604289,...,1.000000,0.984915,0.923351,0.845071,0.757171,0.666369,0.576111,0.513544,0.474153,0.445170
6,0.784894,0.698024,0.625978,0.560636,0.497718,0.439149,0.380639,0.327857,0.285803,0.249527,...,0.561523,0.684073,0.776264,0.859359,0.932519,0.982343,1.000000,0.965750,0.876089,0.754633
7,0.819066,0.731793,0.656563,0.598972,0.530244,0.478081,0.423124,0.369529,0.322622,0.280661,...,0.563682,0.667353,0.760985,0.838543,0.916914,0.973212,1.000000,0.980370,0.908779,0.809878
8,0.714624,0.630896,0.560813,0.498146,0.439424,0.373653,0.325925,0.273260,0.238296,0.209361,...,0.665409,0.776365,0.860848,0.924485,0.971307,1.000000,0.992972,0.942162,0.854596,0.733789


In [15]:
# sort by 'closeness' of hour with the highest temperature
site_ids_argmax_maxtemp = pd.Series(np.argmax(df_2d.values, axis=1)).sort_values().index
site_ids_argmax_maxtemp

Int64Index([1, 5, 12, 0, 8, 3, 6, 7, 11, 14, 15, 9, 13, 4, 10, 2], dtype='int64')

In [25]:
# assuming (1,5,12) tuple has the most correct temp peaks at 14:00
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets

0     5
1     0
2     9
3     6
4     8
5     0
6     6
7     6
8     5
9     7
10    8
11    6
12    0
13    7
14    6
15    6
dtype: int64

In [24]:
df_2d.values.argmax(axis=1)

array([19, 14, 23, 20, 22, 14, 20, 20, 19, 21, 22, 20, 14, 21, 20, 20],
      dtype=int64)

In [27]:
temp_skeleton['offset'] = temp_skeleton.site_id.map(site_ids_offsets)

# add offset
temp_skeleton['timestamp_aligned'] = (
    temp_skeleton.timestamp 
    - pd.to_timedelta(temp_skeleton.offset, unit='H')
)

temp_skeleton.head()

,site_id,timestamp,air_temperature,temp_rank,offset,timestamp_aligned
0,0,2016-01-01 00:00:00,25.000000,16.0,5,2015-12-31 19:00:00
1,0,2016-01-01 01:00:00,24.400000,15.0,5,2015-12-31 20:00:00
2,0,2016-01-01 02:00:00,22.799999,13.0,5,2015-12-31 21:00:00
3,0,2016-01-01 03:00:00,21.100000,9.0,5,2015-12-31 22:00:00
4,0,2016-01-01 04:00:00,20.000000,3.5,5,2015-12-31 23:00:00


In [28]:
temp_skeleton.to_csv('temp_skeleton.csv',index=False)